In [1]:
import uuid
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, JSON, Table
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from sqlalchemy.dialects.postgresql import UUID

Base = declarative_base()

# Association tables for many-to-many relationships
requirement_function_association = Table(
    'requirement_function', Base.metadata,
    Column('requirement_id', Integer, ForeignKey('requirements.id')),
    Column('function_id', Integer, ForeignKey('functions.id'))
)

function_physical_association = Table(
    'function_physical', Base.metadata,
    Column('function_id', Integer, ForeignKey('functions.id')),
    Column('physical_id', Integer, ForeignKey('physicals.id'))
)

# Self-referential association tables for sub-requirements, sub-functions, and sub-physicals
sub_requirement_association = Table(
    'sub_requirement', Base.metadata,
    Column('parent_id', Integer, ForeignKey('requirements.id')),
    Column('child_id', Integer, ForeignKey('requirements.id'))
)

sub_function_association = Table(
    'sub_function', Base.metadata,
    Column('parent_id', Integer, ForeignKey('functions.id')),
    Column('child_id', Integer, ForeignKey('functions.id'))
)

sub_physical_association = Table(
    'sub_physical', Base.metadata,
    Column('parent_id', Integer, ForeignKey('physicals.id')),
    Column('child_id', Integer, ForeignKey('physicals.id'))
)

# Define the top-level Component model with UUID
class Component(Base):
    __tablename__ = 'components'

    id = Column(Integer, primary_key=True)
    uuid = Column(UUID(as_uuid=True), default=uuid.uuid4, unique=True, nullable=False)  # UUID field
    name = Column(String, nullable=False)
    description = Column(String, nullable=True)

    # One-to-many relationship with requirements
    requirements = relationship("Requirement", back_populates="component")

    def __repr__(self):
        return f"<Component(id={self.id}, uuid={self.uuid}, name='{self.name}')>"

# Define the Requirement model with UUID and JSON data
class Requirement(Base):
    __tablename__ = 'requirements'

    id = Column(Integer, primary_key=True)
    uuid = Column(UUID(as_uuid=True), default=uuid.uuid4, unique=True, nullable=False)  # UUID field
    component_id = Column(Integer, ForeignKey('components.id'), nullable=True)  # Made nullable for sub-requirements
    data = Column(JSON, nullable=False)  # Store Requirement as JSON

    component = relationship("Component", back_populates="requirements")
    functions = relationship("Function", secondary=requirement_function_association, back_populates="requirements")
    
    # Self-referential relationship for sub-requirements
    sub_requirements = relationship(
        "Requirement", secondary=sub_requirement_association,
        primaryjoin=id == sub_requirement_association.c.parent_id,
        secondaryjoin=id == sub_requirement_association.c.child_id,
        backref="parent_requirements"
    )

    def __repr__(self):
        return f"<Requirement(id={self.id}, uuid={self.uuid}, data={self.data})>"

# Define the Function model with UUID and JSON data
class Function(Base):
    __tablename__ = 'functions'

    id = Column(Integer, primary_key=True)
    uuid = Column(UUID(as_uuid=True), default=uuid.uuid4, unique=True, nullable=False)  # UUID field
    data = Column(JSON, nullable=False)  # Store Function as JSON

    requirements = relationship("Requirement", secondary=requirement_function_association, back_populates="functions")
    physicals = relationship("Physical", secondary=function_physical_association, back_populates="functions")
    
    # Self-referential relationship for sub-functions
    sub_functions = relationship(
        "Function", secondary=sub_function_association,
        primaryjoin=id == sub_function_association.c.parent_id,
        secondaryjoin=id == sub_function_association.c.child_id,
        backref="parent_functions"
    )

    def __repr__(self):
        return f"<Function(id={self.id}, uuid={self.uuid}, data={self.data})>"

# Define the Physical model with UUID and JSON data
class Physical(Base):
    __tablename__ = 'physicals'

    id = Column(Integer, primary_key=True)
    uuid = Column(UUID(as_uuid=True), default=uuid.uuid4, unique=True, nullable=False)  # UUID field
    data = Column(JSON, nullable=False)  # Store Physical as JSON

    functions = relationship("Function", secondary=function_physical_association, back_populates="physicals")
    
    # Self-referential relationship for sub-physical elements
    sub_physicals = relationship(
        "Physical", secondary=sub_physical_association,
        primaryjoin=id == sub_physical_association.c.parent_id,
        secondaryjoin=id == sub_physical_association.c.child_id,
        backref="parent_physicals"
    )

    def __repr__(self):
        return f"<Physical(id={self.id}, uuid={self.uuid}, data={self.data})>"

# Database setup
engine = create_engine('sqlite:///db_model_5_test.db')
Base.metadata.create_all(engine)

# Session setup
Session = sessionmaker(bind=engine)
session = Session()


In [2]:
from uuid import UUID

# Existing data
requirements_data = [
    {
        "index": "1",
        "title": "Safety Features",
        "description": "The garage door opener must have a safe and reliable operation, with features such as automatic reversing in case of obstacles, safety sensors, and emergency stop.",
        "uuid": "261c3cf5-974d-4e82-8952-0c26c6e2a304"
    },
    {
        "index": "2",
        "title": "User Interface",
        "description": "The garage door opener must have a user-friendly interface, including a remote control, wall console, and/or smartphone app, that allows users to easily open, close, and monitor the garage door.",
        "uuid": "207c18cd-89bc-4f8f-ae79-67af6fcf8a93"
    }
]

functions_data = [
    {
        "requirement_uuid": "261c3cf5-974d-4e82-8952-0c26c6e2a304",  # Links to the requirement with this UUID
        "description": "Detect obstacles",
        "operation": "Use sensors to detect objects in the path of the door."
    },
    {
        "requirement_uuid": "261c3cf5-974d-4e82-8952-0c26c6e2a304",  # Links to the requirement with this UUID
        "description": "Emergency stop",
        "operation": "Immediately halt the door movement when an obstacle is detected."
    },
    {
        "requirement_uuid": "207c18cd-89bc-4f8f-ae79-67af6fcf8a93",  # Links to the requirement with this UUID
        "description": "Remote control operation",
        "operation": "Allow users to operate the door remotely."
    },
    {
        "requirement_uuid": "207c18cd-89bc-4f8f-ae79-67af6fcf8a93",  # Links to the requirement with this UUID
        "description": "Smartphone app integration",
        "operation": "Provide a mobile app for controlling and monitoring the door."
    }
]

# Insert Requirements
requirement_map = {}

for req_data in requirements_data:
    # Insert Requirement
    # requirement_uuid = uuid.uuid4() 
    # this will be used when we use llm to retrive the requirements. llm will not give uuid we will create one.
    
    requirement_uuid = UUID(req_data["uuid"])
    
    requirement = Requirement(
        uuid=UUID(req_data["uuid"]),  # Convert string to UUID object
        data={
            "index": req_data["index"],
            # 'uuid':UUID(req_data["uuid"]),
            'uuid':str(requirement_uuid),
            "title": req_data["title"],
            "description": req_data["description"]
        }
    )
    session.add(requirement)
    session.commit()  # Commit to save the requirement

    # Store the requirement in the map using its UUID as the key
    # requirement_map[req_data["uuid"]] = requirement
    
retrieved_requirements = session.query(Requirement).all()
requirement_map = {req.data["uuid"]: req for req in retrieved_requirements}

# Insert Functions and link them by UUID
for func_data in functions_data:
    # Insert Function
    function = Function(
        data={
            "description": func_data["description"],
            "operation": func_data["operation"]
        }
    )
    session.add(function)
    session.commit()  # Commit to save the function

    # Link Function to Requirement by UUID
    requirement = requirement_map.get(func_data["requirement_uuid"])
    if requirement:
        requirement.functions.append(function)
        session.commit()

print("Requirements and linked functions added successfully!")


Requirements and linked functions added successfully!


In [3]:
from sqlalchemy.orm import joinedload

# Query requirements with functions using a join
requirements_with_functions = session.query(Requirement).options(joinedload(Requirement.functions)).all()

for requirement in requirements_with_functions:
    print(f"Requirement: {requirement.data['title']}")
    for function in requirement.functions:
        print(f"  - Function: {function.data['description']}, Operation: {function.data['operation']}")


Requirement: Safety Features
  - Function: Detect obstacles, Operation: Use sensors to detect objects in the path of the door.
  - Function: Emergency stop, Operation: Immediately halt the door movement when an obstacle is detected.
Requirement: User Interface
  - Function: Remote control operation, Operation: Allow users to operate the door remotely.
  - Function: Smartphone app integration, Operation: Provide a mobile app for controlling and monitoring the door.


v_1 Public Methods for CRUD opertions on REFL

In [4]:
def add_requirement(session, component_id, data):
    """
    Adds a new requirement to the database.
    """
    # Validate component existence
    component = session.query(Component).filter_by(uuid=component_id).first()
    if not component:
        raise ValueError(f"No component found with ID {component_id}")

    requirement = Requirement(component_id=component.id, data=data)
    session.add(requirement)
    session.commit()
    return requirement

def add_sub_requirement(session, parent_requirement_id, data):
    """
    Adds a new sub-requirement to a parent requirement.
    """
    parent_requirement = session.query(Requirement).filter_by(uuid=parent_requirement_id).first()
    if not parent_requirement:
        raise ValueError(f"No parent requirement found with ID {parent_requirement_id}")

    sub_requirement = Requirement(data=data)
    session.add(sub_requirement)
    session.commit()  # Commit here to ensure the sub-requirement gets an ID

    # Link to the parent requirement
    parent_requirement.sub_requirements.append(sub_requirement)
    session.commit()
    return sub_requirement

def add_function(session, data):
    """
    Adds a new function to the database.
    """
    function = Function(data=data)
    session.add(function)
    session.commit()
    return function

def add_sub_function(session, parent_function_id, data):
    """
    Adds a new sub-function to a parent function.
    """
    parent_function = session.query(Function).filter_by(uuid=parent_function_id).first()
    if not parent_function:
        raise ValueError(f"No parent function found with ID {parent_function_id}")

    sub_function = Function(data=data)
    session.add(sub_function)
    session.commit()  # Commit to generate the sub-function ID

    # Link to the parent function
    parent_function.sub_functions.append(sub_function)
    session.commit()
    return sub_function

def add_physical(session, data):
    """
    Adds a new physical entity to the database.
    """
    physical = Physical(data=data)
    session.add(physical)
    session.commit()
    return physical

def add_sub_physical(session, parent_physical_id, data):
    """
    Adds a new sub-physical entity to a parent physical entity.
    """
    parent_physical = session.query(Physical).filter_by(uuid=parent_physical_id).first()
    if not parent_physical:
        raise ValueError(f"No parent physical found with ID {parent_physical_id}")

    sub_physical = Physical(data=data)
    session.add(sub_physical)
    session.commit()  # Commit to generate the sub-physical ID

    # Link to the parent physical
    parent_physical.sub_physicals.append(sub_physical)
    session.commit()
    return sub_physical


use case of public functions

In [5]:

requirement = add_requirement(session, component_id=1, data={"title": "Requirement 1", "description": "Top-level requirement"})
print(requirement)


sub_requirement = add_sub_requirement(session, parent_requirement_id=requirement.id, data={"title": "Sub-Requirement 1", "description": "Child of Requirement 1"})
print(sub_requirement)


function = add_function(session, data={"name": "Function 1", "description": "Top-level function"})
print(function)


sub_function = add_sub_function(session, parent_function_id=function.id, data={"name": "Sub-Function 1", "description": "Child of Function 1"})
print(sub_function)


physical = add_physical(session, data={"name": "Physical 1", "description": "Top-level physical entity"})
print(physical)


sub_physical = add_sub_physical(session, parent_physical_id=physical.id, data={"name": "Sub-Physical 1", "description": "Child of Physical 1"})
print(sub_physical)


ValueError: No component found with ID 1

v_2 CRUD operation public functions

In [2]:
# Function to add a new component
def add_component(session, name, description=None):
    component = Component(name=name, description=description)
    session.add(component)
    session.commit()
    return component

# Function to add a new requirement
def add_requirement(session, component_id, data):
    requirement = Requirement(component_id=component_id, data=data)
    session.add(requirement)
    session.commit()
    return requirement

# Function to add a sub-requirement
def add_sub_requirement(session, parent_requirement_id, child_requirement_id):
    parent_requirement = session.query(Requirement).get(parent_requirement_id)
    child_requirement = session.query(Requirement).get(child_requirement_id)
    parent_requirement.sub_requirements.append(child_requirement)
    session.commit()
    return parent_requirement

# Function to add a new function
def add_function(session, data):
    function = Function(data=data)
    session.add(function)
    session.commit()
    return function

# Function to associate a function with a requirement
def associate_function_with_requirement(session, requirement_id, function_id):
    requirement = session.query(Requirement).get(requirement_id)
    function = session.query(Function).get(function_id)
    requirement.functions.append(function)
    session.commit()
    return requirement

# Function to add a sub-function
def add_sub_function(session, parent_function_id, child_function_id):
    parent_function = session.query(Function).get(parent_function_id)
    child_function = session.query(Function).get(child_function_id)
    parent_function.sub_functions.append(child_function)
    session.commit()
    return parent_function

# Function to add a new physical element
def add_physical(session, data):
    physical = Physical(data=data)
    session.add(physical)
    session.commit()
    return physical

# Function to associate a physical element with a function
def associate_physical_with_function(session, function_id, physical_id):
    function = session.query(Function).get(function_id)
    physical = session.query(Physical).get(physical_id)
    function.physicals.append(physical)
    session.commit()
    return function

# Function to add a sub-physical element
def add_sub_physical(session, parent_physical_id, child_physical_id):
    parent_physical = session.query(Physical).get(parent_physical_id)
    child_physical = session.query(Physical).get(child_physical_id)
    parent_physical.sub_physicals.append(child_physical)
    session.commit()
    return parent_physical


Read Operations

In [18]:
# Function to get a component by ID or UUID
def get_component(session, component_id=None, component_uuid=None):
    if component_id:
        return session.query(Component).get(component_id)
    if component_uuid:
        return session.query(Component).filter_by(uuid=component_uuid).first()
    return None

# Function to get all components
def get_all_components(session):
    return session.query(Component).all()

# Function to get a requirement by ID or UUID
def get_requirement(session, requirement_id=None, requirement_uuid=None):
    if requirement_id:
        return session.query(Requirement).get(requirement_id)
    if requirement_uuid:
        return session.query(Requirement).filter_by(uuid=requirement_uuid).first()
    return None

# Function to get all requirements for a component
def get_requirements_for_component(session, component_id):
    return session.query(Requirement).filter_by(component_id=component_id).all()

# Function to get a function by ID or UUID
def get_function(session, function_id=None, function_uuid=None):
    if function_id:
        return session.query(Function).get(function_id)
    if function_uuid:
        return session.query(Function).filter_by(uuid=function_uuid).first()
    return None

# Function to get a physical element by ID or UUID
def get_physical(session, physical_id=None, physical_uuid=None):
    if physical_id:
        return session.query(Physical).get(physical_id)
    if physical_uuid:
        return session.query(Physical).filter_by(uuid=physical_uuid).first()
    return None


update operations

In [19]:
# Function to update a component
def update_component(session, component_id, name=None, description=None):
    component = session.query(Component).get(component_id)
    if not component:
        return None
    if name:
        component.name = name
    if description:
        component.description = description
    session.commit()
    return component

# Function to update a requirement
def update_requirement(session, requirement_id, data=None):
    requirement = session.query(Requirement).get(requirement_id)
    if not requirement:
        return None
    if data:
        requirement.data = data
    session.commit()
    return requirement

# Function to update a function
def update_function(session, function_id, data=None):
    function = session.query(Function).get(function_id)
    if not function:
        return None
    if data:
        function.data = data
    session.commit()
    return function

# Function to update a physical element
def update_physical(session, physical_id, data=None):
    physical = session.query(Physical).get(physical_id)
    if not physical:
        return None
    if data:
        physical.data = data
    session.commit()
    return physical


Delete operations

In [20]:
# Function to delete a component
def delete_component(session, component_id):
    component = session.query(Component).get(component_id)
    if not component:
        return False
    session.delete(component)
    session.commit()
    return True

# Function to delete a requirement
def delete_requirement(session, requirement_id):
    requirement = session.query(Requirement).get(requirement_id)
    if not requirement:
        return False
    session.delete(requirement)
    session.commit()
    return True

# Function to delete a function
def delete_function(session, function_id):
    function = session.query(Function).get(function_id)
    if not function:
        return False
    session.delete(function)
    session.commit()
    return True

# Function to delete a physical element
def delete_physical(session, physical_id):
    physical = session.query(Physical).get(physical_id)
    if not physical:
        return False
    session.delete(physical)
    session.commit()
    return True


Usage

The functions provided earlier primarily cover the **Create** and some **Update** operations for your schema. However, they do not cover **Read** (retrieving data) or **Delete** operations comprehensively. To make them sufficient for all CRUD operations, we need to expand the set of functions as follows:

---

### **Create**
Already covered in the earlier implementation.

---

### **Read**

```python
# Function to get a component by ID or UUID
def get_component(session, component_id=None, component_uuid=None):
    if component_id:
        return session.query(Component).get(component_id)
    if component_uuid:
        return session.query(Component).filter_by(uuid=component_uuid).first()
    return None

# Function to get all components
def get_all_components(session):
    return session.query(Component).all()

# Function to get a requirement by ID or UUID
def get_requirement(session, requirement_id=None, requirement_uuid=None):
    if requirement_id:
        return session.query(Requirement).get(requirement_id)
    if requirement_uuid:
        return session.query(Requirement).filter_by(uuid=requirement_uuid).first()
    return None

# Function to get all requirements for a component
def get_requirements_for_component(session, component_id):
    return session.query(Requirement).filter_by(component_id=component_id).all()

# Function to get a function by ID or UUID
def get_function(session, function_id=None, function_uuid=None):
    if function_id:
        return session.query(Function).get(function_id)
    if function_uuid:
        return session.query(Function).filter_by(uuid=function_uuid).first()
    return None

# Function to get a physical element by ID or UUID
def get_physical(session, physical_id=None, physical_uuid=None):
    if physical_id:
        return session.query(Physical).get(physical_id)
    if physical_uuid:
        return session.query(Physical).filter_by(uuid=physical_uuid).first()
    return None
```

---

### **Update**

```python
# Function to update a component
def update_component(session, component_id, name=None, description=None):
    component = session.query(Component).get(component_id)
    if not component:
        return None
    if name:
        component.name = name
    if description:
        component.description = description
    session.commit()
    return component

# Function to update a requirement
def update_requirement(session, requirement_id, data=None):
    requirement = session.query(Requirement).get(requirement_id)
    if not requirement:
        return None
    if data:
        requirement.data = data
    session.commit()
    return requirement

# Function to update a function
def update_function(session, function_id, data=None):
    function = session.query(Function).get(function_id)
    if not function:
        return None
    if data:
        function.data = data
    session.commit()
    return function

# Function to update a physical element
def update_physical(session, physical_id, data=None):
    physical = session.query(Physical).get(physical_id)
    if not physical:
        return None
    if data:
        physical.data = data
    session.commit()
    return physical
```

---

### **Delete**

```python
# Function to delete a component
def delete_component(session, component_id):
    component = session.query(Component).get(component_id)
    if not component:
        return False
    session.delete(component)
    session.commit()
    return True

# Function to delete a requirement
def delete_requirement(session, requirement_id):
    requirement = session.query(Requirement).get(requirement_id)
    if not requirement:
        return False
    session.delete(requirement)
    session.commit()
    return True

# Function to delete a function
def delete_function(session, function_id):
    function = session.query(Function).get(function_id)
    if not function:
        return False
    session.delete(function)
    session.commit()
    return True

# Function to delete a physical element
def delete_physical(session, physical_id):
    physical = session.query(Physical).get(physical_id)
    if not physical:
        return False
    session.delete(physical)
    session.commit()
    return True
```

---

### Full CRUD Coverage:
Now the expanded set of functions supports:
1. **Create**: Adding new components, requirements, functions, physical elements, and associations.
2. **Read**: Fetching specific or all entities and related data.
3. **Update**: Modifying existing entities.
4. **Delete**: Removing entities.

---

### Example Usage:

1. **Read a Component**:
   ```python
   component = get_component(session, component_uuid=uuid.UUID('446c9b02-27dd-4ea0-bd6f-6fa50f81ee21'))
   print(component)
   ```

2. **Update a Requirement**:
   ```python
   updated_requirement = update_requirement(session, requirement_id=1, data={"type": "updated", "details": "Updated details"})
   print(updated_requirement)
   ```

3. **Delete a Function**:
   ```python
   success = delete_function(session, function_id=1)
   print("Deleted" if success else "Not found")
   ```

These functions should be sufficient for all CRUD operations in your schema.

In [3]:
component = add_component(session, name="Engine", description="Car engine component")
print(component)


<Component(id=1, uuid=ffe6d0cd-aeab-4838-8918-0cf729ed5b3b, name='Engine')>


In [4]:
requirement = add_requirement(session, component_id=component.id, data={"type": "safety", "details": "Must be heat resistant"})
print(requirement)


<Requirement(id=1, uuid=3bdebb96-c843-4504-b848-f612862a6d9b, data={'type': 'safety', 'details': 'Must be heat resistant'})>


In [ ]:
another_requirement = add_requirement(session, component_id=component.id, data={"type": "legal", "details": "Must be legal resistant"})
print(requirement)

sub_requirement = add_sub_requirement(session, parent_requirement_id=requirement.id, child_requirement_id=another_requirement.id)
print(sub_requirement)


In [21]:
# print(component.requirements[0].sub_requirements)

requirement = session.query(Requirement).filter_by(id = 1).first()

print(requirement)

<Requirement(id=1, uuid=3bdebb96-c843-4504-b848-f612862a6d9b, data={'type': 'safety', 'details': 'Must be heat resistant'})>


In [26]:
comp = session.query(Component).filter_by(uuid = uuid.UUID("ffe6d0cd-aeab-4838-8918-0cf729ed5b3b")).first()
print(comp)

<Component(id=1, uuid=ffe6d0cd-aeab-4838-8918-0cf729ed5b3b, name='Engine')>


In [ ]:
import uuid

# Convert the UUID string to a uuid.UUID object
requirement_uuid = uuid.UUID('446c9b02-27dd-4ea0-bd6f-6fa50f81ee21')

# Query the database for the requirement with the given UUID
requirement = session.query(Requirement).filter_by(uuid=requirement_uuid).first()

# Print the result
print(requirement)


Here’s a set of Python functions to interact with the schema and perform CRUD operations to add components, requirements, functions, sub-requirements, sub-functions, and sub-physical elements. These functions use the SQLAlchemy ORM session to perform operations.

```python
# Function to add a new component
def add_component(session, name, description=None):
    component = Component(name=name, description=description)
    session.add(component)
    session.commit()
    return component

# Function to add a new requirement
def add_requirement(session, component_id, data):
    requirement = Requirement(component_id=component_id, data=data)
    session.add(requirement)
    session.commit()
    return requirement

# Function to add a sub-requirement
def add_sub_requirement(session, parent_requirement_id, child_requirement_id):
    parent_requirement = session.query(Requirement).get(parent_requirement_id)
    child_requirement = session.query(Requirement).get(child_requirement_id)
    parent_requirement.sub_requirements.append(child_requirement)
    session.commit()
    return parent_requirement

# Function to add a new function
def add_function(session, data):
    function = Function(data=data)
    session.add(function)
    session.commit()
    return function

# Function to associate a function with a requirement
def associate_function_with_requirement(session, requirement_id, function_id):
    requirement = session.query(Requirement).get(requirement_id)
    function = session.query(Function).get(function_id)
    requirement.functions.append(function)
    session.commit()
    return requirement

# Function to add a sub-function
def add_sub_function(session, parent_function_id, child_function_id):
    parent_function = session.query(Function).get(parent_function_id)
    child_function = session.query(Function).get(child_function_id)
    parent_function.sub_functions.append(child_function)
    session.commit()
    return parent_function

# Function to add a new physical element
def add_physical(session, data):
    physical = Physical(data=data)
    session.add(physical)
    session.commit()
    return physical

# Function to associate a physical element with a function
def associate_physical_with_function(session, function_id, physical_id):
    function = session.query(Function).get(function_id)
    physical = session.query(Physical).get(physical_id)
    function.physicals.append(physical)
    session.commit()
    return function

# Function to add a sub-physical element
def add_sub_physical(session, parent_physical_id, child_physical_id):
    parent_physical = session.query(Physical).get(parent_physical_id)
    child_physical = session.query(Physical).get(child_physical_id)
    parent_physical.sub_physicals.append(child_physical)
    session.commit()
    return parent_physical
```

### Example Usage:

1. **Add a Component**:
   ```python
   component = add_component(session, name="Engine", description="Car engine component")
   print(component)
   ```

2. **Add a Requirement**:
   ```python
   requirement = add_requirement(session, component_id=component.id, data={"type": "safety", "details": "Must be heat resistant"})
   print(requirement)
   ```

3. **Add a Sub-Requirement**:
   ```python
   sub_requirement = add_sub_requirement(session, parent_requirement_id=requirement.id, child_requirement_id=another_requirement.id)
   print(sub_requirement)
   ```

4. **Add a Function**:
   ```python
   function = add_function(session, data={"type": "operation", "details": "Rotate at 3000 RPM"})
   print(function)
   ```

5. **Associate Function with Requirement**:
   ```python
   updated_requirement = associate_function_with_requirement(session, requirement_id=requirement.id, function_id=function.id)
   print(updated_requirement)
   ```

6. **Add a Sub-Function**:
   ```python
   sub_function = add_sub_function(session, parent_function_id=function.id, child_function_id=another_function.id)
   print(sub_function)
   ```

7. **Add a Physical Element**:
   ```python
   physical = add_physical(session, data={"type": "material", "details": "Aluminum"})
   print(physical)
   ```

8. **Associate Physical Element with Function**:
   ```python
   updated_function = associate_physical_with_function(session, function_id=function.id, physical_id=physical.id)
   print(updated_function)
   ```

9. **Add a Sub-Physical Element**:
   ```python
   sub_physical = add_sub_physical(session, parent_physical_id=physical.id, child_physical_id=another_physical.id)
   print(sub_physical)
   ```

### Notes:
- Each function assumes valid IDs are passed; ensure to handle exceptions for invalid queries or missing entities.
- Use `session.query(<Model>).all()` to inspect the current database state.

uuid is already in the json

In [ ]:
from uuid import UUID

# Existing data
requirements_data = [
    {
        "index": "1",
        "title": "Safety Features",
        "description": "The garage door opener must have a safe and reliable operation, with features such as automatic reversing in case of obstacles, safety sensors, and emergency stop.",
        "uuid": "261c3cf5-974d-4e82-8952-0c26c6e2a304"
    },
    {
        "index": "2",
        "title": "User Interface",
        "description": "The garage door opener must have a user-friendly interface, including a remote control, wall console, and/or smartphone app, that allows users to easily open, close, and monitor the garage door.",
        "uuid": "207c18cd-89bc-4f8f-ae79-67af6fcf8a93"
    }
]

functions_data = [
    {
        "requirement_uuid": "261c3cf5-974d-4e82-8952-0c26c6e2a304",  # Links to the requirement with this UUID
        "description": "Detect obstacles",
        "operation": "Use sensors to detect objects in the path of the door."
    },
    {
        "requirement_uuid": "261c3cf5-974d-4e82-8952-0c26c6e2a304",  # Links to the requirement with this UUID
        "description": "Emergency stop",
        "operation": "Immediately halt the door movement when an obstacle is detected."
    },
    {
        "requirement_uuid": "207c18cd-89bc-4f8f-ae79-67af6fcf8a93",  # Links to the requirement with this UUID
        "description": "Remote control operation",
        "operation": "Allow users to operate the door remotely."
    },
    {
        "requirement_uuid": "207c18cd-89bc-4f8f-ae79-67af6fcf8a93",  # Links to the requirement with this UUID
        "description": "Smartphone app integration",
        "operation": "Provide a mobile app for controlling and monitoring the door."
    }
]

# Insert Requirements
requirement_map = {}

for req_data in requirements_data:
    # Insert Requirement
    requirement_uuid = uuid.uuid4()
    requirement = Requirement(
        uuid=UUID(req_data["uuid"]),  # Convert string to UUID object
        data={
            "index": req_data["index"],
            # 'uuid':UUID(req_data["uuid"]),
            'uuid':str(requirement_uuid),
            "title": req_data["title"],
            "description": req_data["description"]
        }
    )
    session.add(requirement)
    session.commit()  # Commit to save the requirement

    # Store the requirement in the map using its UUID as the key
    requirement_map[req_data["uuid"]] = requirement

# Insert Functions and link them by UUID
for func_data in functions_data:
    # Insert Function
    function = Function(
        data={
            "description": func_data["description"],
            "operation": func_data["operation"]
        }
    )
    session.add(function)
    session.commit()  # Commit to save the function

    # Link Function to Requirement by UUID
    requirement = requirement_map.get(func_data["requirement_uuid"])
    if requirement:
        requirement.functions.append(function)
        session.commit()

print("Requirements and linked functions added successfully!")
